# Query datalakes on S3 with Amazon Athena

Visit the followint AWS blogpost for more detailed instructions: https://aws.amazon.com/blogs/machine-learning/run-sql-queries-from-your-sagemaker-notebooks-using-amazon-athena/

### Install dependencies.
PyAthena is a Python DB API 2.0 (PEP 249) compliant client for the Amazon Athena JDBC driver.

In [1]:
import sys
!{sys.executable} -m pip install PyAthena

    100% |████████████████████████████████| 5.8MB 1.5MB/s ta 0:00:01    35% |███████████▍                    | 2.1MB 1.9MB/s eta 0:00:03    70% |██████████████████████▋         | 4.1MB 2.2MB/s eta 0:00:01
    100% |████████████████████████████████| 133kB 1.7MB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/83/94/7179c3832a6d45b266ddb2aac329e101367fbdb11f425f13771d27f225bb/jmespath-0.9.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/41/17/c62faccbfbd163c7f57f3844689e3a78bae1f403648a6afb1d0866d87fbb/python_dateutil-2.8.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/16/8a/1fc3dba0c4923c2a76e1ff0d52b305c44606da63f718d14d3231e21c51b0/s3transfer-0.2.1-py2.py3-none-any.whl
  Found existing installation: python-dateutil 2.8.1
    Uninstalling python-dateutil-2.8.1:
      Successfully uninstalled python-dateutil-2.8.1


### Connect to Athena and run SQL queries against your datalake on S3.

In [7]:
from pyathena import connect
import pandas as pd
conn = connect(s3_staging_dir='s3://athena-staging-fulu',
               region_name='eu-west-1')

df = pd.read_sql("SELECT * FROM udacity.accidents limit 10;", conn)
df

,accident_id,datetime,city,state_code,state,airport_code,start_time,end_time,timezone,severity,temperature,year,month
0,A-426,2016-03-01,Columbus,OH,OH,KTZR,2016-03-01 05:05:42,2016-03-01 05:35:42,US/Eastern,2,34.0,2016,3
1,A-427,2016-03-01,Miamisburg,OH,OH,KMGY,2016-03-01 06:14:44,2016-03-01 06:44:44,US/Eastern,2,37.9,2016,3
2,A-428,2016-03-01,Dayton,OH,OH,KDAY,2016-03-01 07:30:18,2016-03-01 08:00:18,US/Eastern,2,37.0,2016,3
3,A-429,2016-03-01,Columbus,OH,OH,KTZR,2016-03-01 07:31:53,2016-03-01 08:01:53,US/Eastern,2,35.6,2016,3
4,A-430,2016-03-01,Columbus,OH,OH,KCMH,2016-03-01 07:46:03,2016-03-01 08:16:03,US/Eastern,2,37.0,2016,3
5,A-431,2016-03-01,Columbus,OH,OH,KTZR,2016-03-01 08:16:30,2016-03-01 08:46:30,US/Eastern,3,33.8,2016,3
6,A-432,2016-03-01,Fairborn,OH,OH,KFFO,2016-03-01 10:06:47,2016-03-01 10:36:47,US/Eastern,2,46.6,2016,3
7,A-433,2016-03-01,Batavia,OH,OH,KI69,2016-03-01 10:19:48,2016-03-01 10:49:48,US/Eastern,2,54.1,2016,3
8,A-434,2016-03-01,Milford,OH,OH,KI69,2016-03-01 10:41:37,2016-03-01 11:11:37,US/Eastern,2,54.3,2016,3
9,A-435,2016-03-01,Sardinia,OH,OH,KI69,2016-03-01 11:12:57,2016-03-01 11:42:57,US/Eastern,2,53.6,2016,3


### Example queries

In [12]:
df = pd.read_sql("SELECT COUNT(*) FROM udacity.accidents WHERE year='2016' AND month='5';", conn)
df

,_col0
0,3760
